**TODO**
* get rid of requests we don't use
* annotate the code to explain what we're doing and why [tutorial!]
* have to explain what broad categories we wanted to analyze and why
    * clean up our doc about what categories we want to use and add it to the github, explain why we are only using certain [un] categories for now (most bang for our buck)

In [44]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import datetime
import re

In [62]:
category_links_df = pd.read_csv('category-links.csv')
category_links_df

resolution_urls = []
visited_categories = []

In [61]:
first_page = requests.get(category_links_df["link"][0])
if first_page.status_code != 200:
    raise Exception("Something went wrong loading category: " + category + ", error code: " + first_page.status_code)
    
html = first_page.text
soup = BeautifulSoup(html, 'html.parser')
link_divs = soup.find_all("div", {"class": "moreinfo"}) #divs with all the links inside them

#page_count = NEED TO GET THIS FROM BEAUTIFUL SOUP
 
for page in page_count:
    
    for div in soup.find_all("div", {"class": "moreinfo"}):
        for span in div.find("span"):  
            #this only gets the first span which we want -> there are two but the second span is irrelevant
            
            #print(span)
            a = span.find_all('a', {"class": "moreinfo"})
            print(a) #returning None ???
            #print(a["href"], a.get_text())
            
            #having trouble getting the a's from the span tag? need to get href's from the a tag? 

            
            #link_suffix = need to get the end link from the href inside the a tag
            
            resolution_urls.append('https://digitallibrary.un.org' + link_suffix)
            
    #load the next page
    


            



    

<a class="moreinfo" href="/record/3951465?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3951464?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3951463?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3951462?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3951461?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3951460?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3951459?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3951456?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3894869?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3894868?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3894866?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3894848?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3894847?ln=en">Detailed record</a>
[]
<a class="moreinfo" href="/record/3894846?ln=en">Detailed record

In [ ]:
try:
    arr.append(dangerous_could_fail_func)
    
except:
    # ignore the error lol
    # print oops died
# write arr to the file